In [67]:
# -*- coding: UTF-8 -*-

In [68]:
pip install docx2pdf

Note: you may need to restart the kernel to use updated packages.


In [69]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time
import re

from docxtpl import DocxTemplate
from docxcompose.composer import Composer
from docx import Document
from docx2pdf import convert

In [70]:
df = pd.read_excel('data/probs.xlsx',dtype=str)
name_file_template_doc = 'data/Шаблон профессиональной пробы 18.06'

In [71]:
df

,ID,Время создания,Профессиональная среда,Наименование профессионального направления,ФИО автора программы,Должность/звание автора программы,Регион,Город,Электронная почта автора,Контактный телефон,...,Постановка цели и задачи в рамках пробы.,"Демонстрация итогового результата, продукта.",Подробная инструкция по выполнению задания.,Рекомендации для наставника по организации процесса выполнения задания.,Критерии успешного выполнения задания.,"Рекомендации для наставника по контролю результата, процедуре оценки.",Вопросы для рефлексии обучающихся.,Введите инфраструктурный лист так как указано выше,"Введите дополнительные источники литературы, ссылки на полезную информацию, ссылки на видео и т.п. \nКаждый пункт должен заканчиваться точкой с запятой","Приложите дополнительные файлы (фотографии, чертежи, схемы и т.п.) если они используются в профпробе."
0,1416819968,2023-06-19 11:06:57,техническая,Программист,Иванов Иван Иванович,CDO ЦОПП Бурятия,Забайкальский край,г. Чита,test@test.ru,89503814183,...,"Даже промежуточные результаты, полученные учас...","Даже промежуточные результаты, полученные учас...",- рассказ наставника о конкретной профессионал...,- рассказ наставника о конкретной профессионал...,Площадками по реализации практических занятий ...,Площадками по реализации практических занятий ...,Площадками по реализации практических занятий ...,"Бумага А4;белая,плотность 80г/м²;20 листов;20/...",NaN,NaN
1,1416590916,2023-06-19 04:54:47,Умная,Педагог-психолог,Белькова Мария Родионовна,методист ЦОПП РБ,Республика Бурятия,гор. Улан-Удэ,belykova85@mail.ru,89021626856,...,"Задача школьников состоит в том, чтобы помочь ...",Участники профпробы должны - разработать план ...,"Шаг 1: Введение. Объяснить школьнику, что это ...","При выполнении задания, наставник может предла...",1. Анализ проблемы: Школьник должен продемонст...,Оценка выполнения задания должна основываться ...,Школьникам необходимо закончить фразы: «Я увид...,Комплект мультимедийного оборудования (проекто...,•\tАфанасьев А. Тайм-менеджмент для детей. Меч...,NaN


In [72]:
df.fillna('error',inplace=True)

In [74]:
df.iloc[:,13:16]

,Возрастная категория участников / 6-7 класс,Возрастная категория участников / 8-9 класс,Возрастная категория участников / 10-11 класс
0,6-7 класс,8-9 класс,error
1,error,error,10-11 класс


In [75]:
df['Возрастная_категория'] = df.iloc[:,13:16].apply(lambda row:';'.join(row),axis=1)

In [76]:
df['Возрастная_категория']=df['Возрастная_категория'].apply(lambda x:re.sub(r'error[;]?','',x)) # очищаем от лишнего

In [77]:
df['Возрастная_категория'] = df['Возрастная_категория'].apply(lambda x:re.sub(r'[;]?$','',x))

In [78]:
df['Возрастная_категория']

0    6-7 класс;8-9 класс
1            10-11 класс
Name: Возрастная_категория, dtype: object

In [79]:
df.iloc[:,16:25]

,"Допустимая нозологическая группа/группы / Нарушение зрения (Слепые, Слабовидяшие)","Допустимая нозологическая группа/группы / Нарушение слуха (Глухие, Слабослышащие)",Допустимая нозологическая группа/группы / Нарушение опорно-двигательного аппарата (НОДА),"Допустимая нозологическая группа/группы / Общие заболевания (нарушение дыхательной системы, пищеварительной, эндокринной систем, сердечно-сосудистой системы и т.д.)",Допустимая нозологическая группа/группы / Тяжёлые нарушение речи,Допустимая нозологическая группа/группы / Нарушение интеллекта (умственная отсталость),Допустимая нозологическая группа/группы / Задержка психического развития (ЗПР) в части педагогической запущенности,Допустимая нозологическая группа/группы / Расстройства аутистического спектра (РАС),Допустимая нозологическая группа/группы / Дети с комплексными нарушениями (со сложными дефектами)
0,"Нарушение зрения (Слепые, Слабовидяшие)",error,error,Общие заболевания (нарушение дыхательной систе...,Тяжёлые нарушение речи,error,Задержка психического развития (ЗПР) в части п...,error,error
1,error,error,Нарушение опорно-двигательного аппарата (НОДА),error,error,error,error,error,error


In [80]:
df['Допустимые_нозологии'] = df.iloc[:,16:25].apply(lambda row:';'.join(row),axis=1)

In [81]:
df['Допустимые_нозологии']=df['Допустимые_нозологии'].apply(lambda x:re.sub(r'error[;]?','',x)) # очищаем от лишнего

In [82]:
df['Допустимые_нозологии'] = df['Допустимые_нозологии'].apply(lambda x:re.sub(r'[;]?$','',x))

In [83]:
df['Допустимые_нозологии']

0    Нарушение зрения (Слепые, Слабовидяшие);Общие ...
1       Нарушение опорно-двигательного аппарата (НОДА)
Name: Допустимые_нозологии, dtype: object

In [84]:
df = df.drop(df.columns[13:16],axis=1) # последовательно удаляем лишние колонки

In [85]:
df = df.drop(df.columns[13:22],axis=1) # последовательно удаляем лишние колонки

In [86]:
lst_name_columns = ['ID','Время_создания','Профессиональная_проба','Наименование_профессионального_направления','ФИО',
       'Должность', 'Регион', 'Город','Электронная_почта', 'Контактный_телефон',
                    'Уровень_сложности','Формат_проведения','Время_проведения',
                    'Спец_условия','Возможность_проведения','Краткое_описание','Перспективы','Навыки_знания','Интересные_факты',
                    'Связь_пробы','Постановка_цели','Демонстрация','Инструкция','Рекомендации_организация','Критерии','Рекомендации_контроль',
                    'Рефлексия','Инфраструктурный_лист','Доп_источники','Доп_файлы','Возрастная_категория','Допустимые_нозологии']

In [88]:
df.columns = lst_name_columns

In [97]:
df.columns

Index(['ID', 'Время_создания', 'Профессиональная_проба',
       'Наименование_профессионального_направления', 'ФИО автора программы',
       'Должность/звание автора программы', 'Регион', 'Город',
       'Электронная почта автора', 'Контактный телефон', 'Уровень_сложности',
       'Формат_проведения', 'Время_проведения', 'Спец_условия',
       'Возможность_проведения', 'Краткое_описание', 'Перспективы',
       'Навыки_знания', 'Интересные_факты', 'Связь_пробы', 'Постановка_цели',
       'Демонстрация', 'Инструкция', 'Рекомендации_организация', 'Критерии',
       'Рекомендации_контроль', 'Рефлексия', 'Инфраструктурный_лист',
       'Доп_источники', 'Доп_файлы', 'Возрастная_категория',
       'Допустимые_нозологии'],
      dtype='object')

In [95]:
#df = df.applymap(lambda x:x.replace("\n", " ")) # удаляем перенос строк
#df = df.applymap(lambda x:x.replace("\r", "")) # удаляем возврат каретки
df = df.applymap(lambda x:x.replace("\u00A0", " ")) # удаляем символ неразрывного пробела
df = df.applymap(lambda x:x.replace("error", "")) # очищаем от слова error меняя на пустую строку

In [96]:
df

,ID,Время_создания,Профессиональная_проба,Наименование_профессионального_направления,ФИО автора программы,Должность/звание автора программы,Регион,Город,Электронная почта автора,Контактный телефон,...,Инструкция,Рекомендации_организация,Критерии,Рекомендации_контроль,Рефлексия,Инфраструктурный_лист,Доп_источники,Доп_файлы,Возрастная_категория,Допустимые_нозологии
0,1416819968,2023-06-19 11:06:57,техническая,Программист,Иванов Иван Иванович,CDO ЦОПП Бурятия,Забайкальский край,г. Чита,test@test.ru,89503814183,...,- рассказ наставника о конкретной профессионал...,- рассказ наставника о конкретной профессионал...,Площадками по реализации практических занятий ...,Площадками по реализации практических занятий ...,Площадками по реализации практических занятий ...,"Бумага А4;белая,плотность 80г/м²;20 листов;20/...",,,6-7 класс;8-9 класс,"Нарушение зрения (Слепые, Слабовидяшие);Общие ..."
1,1416590916,2023-06-19 04:54:47,Умная,Педагог-психолог,Белькова Мария Родионовна,методист ЦОПП РБ,Республика Бурятия,гор. Улан-Удэ,belykova85@mail.ru,89021626856,...,"Шаг 1: Введение. Объяснить школьнику, что это ...","При выполнении задания, наставник может предла...",1. Анализ проблемы: Школьник должен продемонст...,Оценка выполнения задания должна основываться ...,Школьникам необходимо закончить фразы: «Я увид...,Комплект мультимедийного оборудования (проекто...,•\tАфанасьев А. Тайм-менеджмент для детей. Меч...,,10-11 класс,Нарушение опорно-двигательного аппарата (НОДА)


In [92]:
 # Конвертируем датафрейм в список словарей
data = df.to_dict('records')

In [93]:
for idx,row in enumerate(data):
    doc = DocxTemplate(name_file_template_doc)
    context = row
    # print(context)
    doc.render(context)
    doc.save(f'data/{idx}.docx')

PackageNotFoundError: Package not found at 'data/Шаблон профессиональной пробы 18.06'